#### KSU Hourly Weather Data
- need to use hourly values of temp and rh to calculate hourly vpd, then daily vpd

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ksu_0 = pd.read_csv('data/raw/ksu/kansas_hourly_weather/ksu_hourly_weather.csv')
# print(ksu_0.shape)
# ksu_0.head()

1. create a column that calculates vpd per row (hour)
2. create new column that only includes the date and not the hourly timestamp
3. group by days to create vpd for each day 
4. add this to the other daily ksu weather data

In [ ]:
# remove first two rows

ksu_1 = ksu_0.iloc[2:]
# print(ksu_1.shape)
# ksu_1.head(3)

In [ ]:
def calculate_vpd_kpa(temp_avg_c, rh_avg):
    es = (6.11 * np.exp((2500000/461) * (1/273 - 1/(273 + temp_avg_c))))
    vpd = (((100 - rh_avg)/1000) * es)
    return vpd

In [ ]:
ksu_2 = ksu_1.copy()
ksu_2['AirTemperature'] = pd.to_numeric(ksu_2['AirTemperature'], errors='coerce')

In [ ]:
ksu_3 = ksu_2.copy()
ksu_3['RelativeHumidity'] = pd.to_numeric(ksu_3['RelativeHumidity'], errors='coerce')
# print(ksu_3.shape)
# print(ksu_3.dtypes)
# ksu_3.head()

In [ ]:
ksu_4 = ksu_3.copy()
ksu_4['vpd_kpa'] = calculate_vpd_kpa(ksu_4['AirTemperature'], ksu_4['RelativeHumidity'])
# print(ksu_4.shape)
# ksu_4.head()

In [ ]:
vpd_0 = ksu_4[['Timestamp', 'vpd_kpa']]
# vpd_0.head()

In [ ]:
# vpd_0.isnull().sum()

In [ ]:
vpd_1 = vpd_0.dropna(how='any', axis=0)
# print(vpd_1.shape)
# vpd_1.head()

In [ ]:
just_dates = []

for timestamp in vpd_1['Timestamp'].values:
    date = timestamp[:10]
    just_dates.append(date)

In [ ]:
vpd_2 = vpd_1.copy()
vpd_2['date'] = just_dates

# print(vpd_2.shape)
# vpd_2.tail()

In [ ]:
daily_vpd_kpa = vpd_2.groupby(['date'], as_index=False)['vpd_kpa'].mean()
# print(daily_vpd_kpa.shape)
# daily_vpd_kpa.head()

In [ ]:
ksu_daily_0 = pd.read_csv('data/weather/ksu_weather.csv')
# print(ksu_daily_0.shape)
# ksu_daily_0.head()

In [ ]:
added_vpd_0 = ksu_daily_0.merge(daily_vpd_kpa, how='left', left_on='date', right_on='date')
# print(added_vpd_0.shape)
# added_vpd_0.head()

In [ ]:
added_vpd_1 = added_vpd_0.copy()
added_vpd_1['vpd_kpa'] = round(added_vpd_1['vpd_kpa'], 2)
# print(added_vpd_1.shape)
# added_vpd_1.head()

In [ ]:
# print(added_vpd_1.date.min())
# print(added_vpd_1.date.max())

In [ ]:
# looked up dates on DOY calendar 169 - 295
days = [day for day in range(169, 296)]

In [ ]:
added_vpd_2 = added_vpd_1.copy()
added_vpd_2['day_of_year'] = days
print(added_vpd_2.shape)
added_vpd_2.tail(3)

In [ ]:
added_vpd_2.to_csv('data/weather/ksu_weather.csv', index=False)